# 6. Data Preparation (7 Points):

In [11]:
#Importing all relevant packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from matplotlib import rcParams
from sklearn.model_selection import cross_val_score, RepeatedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
import pickle
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

# 6.1 Load the dataset and display the dataframe (2 Points).

In [10]:
#Reading in Data Set and displaying first 5 rows
OSI_g2 = pd.read_csv("../datasets/online_shoppers_intention.csv")

#Converting to Integer
OSI_g2["Weekend"] = OSI_g2["Weekend"].astype(int)
OSI_g2["Revenue"] = OSI_g2["Revenue"].astype(int)

#Adding a numeric version of visitor type and dropping original column
OSI_g2.insert(loc=16, column="VisitorTypeNumeric", value=pd.factorize(OSI_g2['VisitorType'])[0] + 1)
OSI_g2 = OSI_g2.drop('VisitorType', axis=1)

#Renaming columns to remove the underscore
new_col_names_g2 = {
    'Administrative_Duration' : 'AdministrativeDuration',
    'Informational_Duration' : 'InformationalDuration',
    'ProductRelated_Duration' : 'ProductRelatedDuration'
}
OSI_g2.rename(columns=new_col_names_g2, inplace=True)

#Printing data set
OSI_g2.head(6)

,Administrative,AdministrativeDuration,Informational,InformationalDuration,ProductRelated,ProductRelatedDuration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorTypeNumeric,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.0,0.0,Feb,1,1,1,1,1,0,0
1,0,0.0,0,0.0,2,64.000000,0.000000,0.100000,0.0,0.0,Feb,2,2,1,2,1,0,0
2,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.0,0.0,Feb,4,1,9,3,1,0,0
3,0,0.0,0,0.0,2,2.666667,0.050000,0.140000,0.0,0.0,Feb,3,2,2,4,1,0,0
4,0,0.0,0,0.0,10,627.500000,0.020000,0.050000,0.0,0.0,Feb,3,3,1,4,1,1,0
5,0,0.0,0,0.0,19,154.216667,0.015789,0.024561,0.0,0.0,Feb,2,2,1,3,1,0,0


# 6.2 Use describe to provide statistics on the pandas Dataframe (2 Points).

In [9]:
#Using describe function to display statistics on the dataframe
OSI_g2.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType,VisitorTypeNumeric,Weekend,Revenue
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,2.315166,80.818611,0.503569,34.472398,31.731468,1194.746220,0.022191,0.043073,5.889258,0.061427,2.124006,2.357097,3.147364,4.069586,1.151176,0.232603,0.154745
std,3.321784,176.779107,1.270156,140.749294,44.475503,1913.669288,0.048488,0.048597,18.568437,0.198917,0.911325,1.717277,2.401591,4.025169,0.376989,0.422509,0.361676
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,184.137500,0.000000,0.014286,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000,1.000000,0.000000,0.000000
50%,1.000000,7.500000,0.000000,0.000000,18.000000,598.936905,0.003112,0.025156,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000,1.000000,0.000000,0.000000
75%,4.000000,93.256250,0.000000,0.000000,38.000000,1464.157214,0.016813,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000,1.000000,0.000000,0.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000,3.000000,1.000000,1.000000


# 6.3 Split the dataset into a Training set and a Test set. Justify your preferred split (3 Points).

In [12]:
#First, we need to impute missing values before proceeding with train/test
imputer = SimpleImputer(strategy='most_frequent')
OSI_imputed_g2 = pd.DataFrame(imputer.fit_transform(OSI_g2), columns=OSI_g2.columns)

In [16]:
#Setting up our X and Y based on our input variables and our target variable being Revenue
X_g2 = OSI_imputed_g2.loc[:, OSI_imputed_g2.columns != 'Revenue'].to_numpy()
y_g2 = OSI_imputed_g2.iloc[:,-1:].to_numpy()

#Using the train_test_split function with 70/30 split since the dataset is imbalanced.
X_train_g2, X_test_g2, y_train_g2, y_test_g2 = train_test_split(X_g2, y_g2, test_size=0.30, random_state=385)

**Because our data set is imbalanced, we decided to use a 70-30 split, rather than 80-20. This will allow our models to train on a greater percentage of the data set to hopefully include the handful of positives in our target variable.**